In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as plb
import math
import itertools
import pickle

from plotly.offline import init_notebook_mode, iplot, plot
from plotly.graph_objs import *
import colorlover as cl
from itertools import compress
import glob

init_notebook_mode(connected=True)   

%matplotlib inline

In [2]:
def hasDigit(inputString):
    return any(char.isdigit() for char in str(inputString))
npHasDigit = np.vectorize(hasDigit, otypes=[np.bool])

def dataframe_row_with_text(df, col, text):
    row = np.nonzero([text in val for val in df.iloc[:,col].astype(str).str.lower().values])[0]
    return row

def dataframe_col_with_text(df, row, text):
    row = np.nonzero([text in val for val in df.iloc[row,:].astype(str).str.lower().values])[0]
    return row

def dataframe_col_with_text(df, row, text):
    row = np.nonzero([text in val for val in df.iloc[row,:].astype(str).str.lower().values])[0]
    return row

def make_columns_unique(df_columns):
    seen = set()

    for item in df_columns:
        fudge = 1
        newitem = item

        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)

        yield newitem
        seen.add(newitem)
#list(make_columns_unique(m.columns))        
        

In [3]:
sn = []
fname = []
sheetNum = []
sheetCol = []
tailNum = []
color = []
sheet = []
fileWithPath = glob.glob('/Users/tank/Downloads/zerobase/BladeMeasurements/*/*/*.xlsx', recursive=True)

count = 0
wingColor =['red', 'yellow', 'blue', 'black']

for pnFn in fileWithPath:
    
    if (pnFn.split('/')[-1][0] != '~'):
        dSheets = pd.read_excel(pnFn, header=None, sheetname=None)
        for k,v in dSheets.items():
            #print(pnFn,k)

            # error check the dataframe, use only sheets with a color in it
            if (len(v) > 0) & (np.any([val in k.lower() for val in wingColor])):

                # populate list
                fname.append(pnFn)
                ksplit = k.split()
                sheetNum.append(ksplit[0])
                if len(ksplit) > 1:
                    sheetCol.append(ksplit[1])

                # find and parse row
                v.dropna(how='all', inplace=True)    #to drop if all values in the row are nan
                sheet.append(v)
                row = np.nonzero(['tail' in val for val in v.iloc[:,0].astype(str).str.lower().values])[0]
                digitIdx = np.nonzero([hasDigit(val) for val in v.iloc[row[0],:]])
    #             print(pnFn.split('/')[-1])
                tailNum.append(v.iloc[[row[0]],:].dropna(axis=1).iloc[0,:].values[1])
                tailRow = v.iloc[[row[0]],:].dropna(axis=1).iloc[0,:].values[2:]
                idx = npHasDigit(tailRow)
                sn.append(tailRow[idx])
                     
    #            print(v.iloc[[row[0]],:].dropna(axis=1).iloc[0,:].values[2:])
    #           print()
    #           print(v.iloc[row[0],digitIdx])
                count = count + 1

###########################Debug            
#             else:
#                 print(pnFn, '\n', k)
            
#     else:
#         print('******', pnFn)

In [4]:
serial = np.zeros(len(sn))
ii = -1
for temp in sn:
    ii = ii + 1
    if len(temp) == 1:
        val = temp[0]
        if isinstance(val, int):
            serial[ii] = int(val)
        elif (isinstance(val, str)):
            if ('-' in val):
                serial[ii] = int(val.split('-')[-1])
            else:
                serial[ii] = int(val.split(' ')[-1])
    elif len(temp) > 1:
        val = temp[0]
        if isinstance(val, str):
            if ('-' in val):
                serial[ii] = int(val.split('-')[-1])
            elif isinstance(temp[1], str):
                val = temp[1]
                if ('-' in val):
                    serial[ii] = int(val.split('-')[-1])   
        elif isinstance(temp[1], str):
            val = temp[1]
            if ('-' in val):
                serial[ii] = int(val.split('-')[-1])
        
                

In [5]:
# measure = []
# for val in sheet:
#     # get the row with position in it (maybe more than one)
#     postionIdx = dataframe_row_with_text(val, 0, 'position')
#     for ii,row in enumerate(postionIdx):
        
#         # find the columns with Rn where n is an integer
#         idx = np.nonzero(npHasDigit(val.iloc[row,:].values))[0]
#         rIdx = dataframe_col_with_text(val.iloc[[row],idx], 0, 'r' )
        
#         # find the row where measurements stop
#         eIdx = val.index[row:].values
#         endArray = np.nonzero(np.diff(eIdx) != 1)[0]
#         if len(endArray) < 1:
#             end = np.where(val.index.values==eIdx[-1])
#         else:
#             end = np.where(val.index.values==eIdx[endArray[0]])
#         #print(val.iloc[list(np.arange(row,end)),np.insert(idx, idx[0]-1, idx[0]-1)])
#         temp = val.iloc[list(np.arange(row,end[0]+1)),idx]
#         side = 'inboard'
#         if ii>0:
#             side = 'outboard'
#         temp['side'] = side
#         measure.append(temp)
        

In [6]:
measure = []
for ii, val in enumerate(sheet):
    # get the row with position in it (maybe more than one)
    postionIdx = dataframe_row_with_text(val, 0, 'position')
    row = postionIdx[0]
        
    # find the columns with Rn where n is an integer
    idx = np.nonzero(npHasDigit(val.iloc[row,:].values))[0]
    rIdx = dataframe_col_with_text(val.iloc[[row],idx], 0, 'r' )

    # find the row where measurements stop
    eIdx = np.in1d(val.iloc[:,0].values, np.array(['A','B','C','D','E','F','G','H','I','J','K','A\'']))
    first = np.nonzero(eIdx)[0][0]
    eIdx[first-1] = True
    #print(val.iloc[list(np.arange(row,end)),np.insert(idx, idx[0]-1, idx[0]-1)])
    temp = val.iloc[eIdx,rIdx]
    
    # reformat sub dataframe
    m = temp.T

    new_header = m.iloc[0] #grab the first row for the header
    m = m[1:] #take the data less the header row
    m.columns = list(make_columns_unique(new_header)) #set the header row as the df header
    m['sn'] = int(serial[ii])
    m['file'] = fname[ii]
    measure.append(m)


In [567]:
betti = pd.concat(measure)

In [568]:
betti

,A,A',A'_2,A_2,B,B_2,C,C_2,D,D_2,...,I_2,J,J_2,K,K_2,Position,Z,Z,file,sn
1,0.25,0.25,0.45,0.45,0.15905,0.469,0.2216,0.47845,0.152,0.42895,...,0.245,0.4214,0.32345,0.44585,0.3544,R1,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
2,0.25,0.25,0.45,0.45,0.1593,0.4686,0.2217,0.47815,0.1521,0.42885,...,0.24575,0.4209,0.3237,0.44535,0.35435,R2,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
3,0.25,0.24995,,0.45,0.1588,0.4683,0.22155,0.47825,0.1516,0.4285,...,0.2448,0.42135,0.32365,0.4461,0.35465,R3,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
4,0.25,0.25,.0.45,0.45,0.1587,0.46845,0.2215,0.4781,0.1515,0.42825,...,0.2449,0.42135,0.3239,0.4461,0.3549,R4,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
5,0.25,0.25,0.45,0.45,0.15875,0.4686,0.22185,0.47805,0.15185,0.4283,...,0.2448,0.4214,0.3237,0.4463,0.3547,R5,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
6,0.25,0.24995,0.45,0.45,0.159,0.46835,0.22135,0.4783,0.15165,0.42855,...,0.2448,0.421,0.3235,0.44565,0.3545,R6,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
7,0.25,0.25,0.4495,0.45,0.15925,0.4686,0.2213,0.47835,0.15175,0.42845,...,0.24465,0.42065,0.3237,0.44535,0.3548,R7,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
8,0.25,0.24995,0.4495,0.45,0.1587,0.4685,0.2212,0.47815,0.15115,0.42875,...,0.24525,0.4212,0.3236,0.4458,0.35435,R8,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
9,0.25,0.25,0.4495,0.45,0.1587,0.4685,0.2211,0.47825,0.1511,0.42845,...,0.2447,0.42115,0.3236,0.44575,0.3547,R9,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2880
1,0.25,0.25,0.45,0.45,0.161,0.4672,0.2259,0.4778,0.1588,0.4256,...,0.2803,0.43265,0.3528,0.4353,0.3604,R1,NaN,NaN,/Users/tank/Downloads/zerobase/BladeMeasuremen...,2836


In [565]:
fname

['/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515_449.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515_449.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515_449.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasurements/Rotor Blade Data 2/Rucker 011514/Ft Rucker Readings_011515_449.xlsx',
 '/Users/tank/Downloads/zerobase/BladeMeasuremen